In [ ]:
!pip install numpy
!pip install opencv-python
!pip install matplotlib
!pip install gradio

In [4]:
import cv2 as cv
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt

from utils import Cam, Img, IO

In [21]:
class Render:
    @staticmethod
    def display_image():
        return
    @staticmethod
    def image_axis():
        return
    @staticmethod
    def image_cal_marks():
        return

# Main calibration workflow
class CalibrationWorkflow:
    """Main calibration workflow using static helper classes"""
    
    def __init__(self, board_size=(9, 6), square_size=25.0, output_dir="/content/images"):
        self.board_size = board_size
        self.square_size = square_size
        self.output_dir = output_dir
        self.obj_points = []
        self.img_points = []
        self.processed_images = []
        self.calibration_results = None
    
    def process_images(self, files):
        """Process uploaded images and detect chessboard corners"""
        imgs = IO.get_images()
        if not files and not imgs:
            return "No files uploaded!"

        IO.store_images(files)
        imgs = IO.get_images()

        opoints, ipoints = Img.find_chessboard_corners(imgs)

        ret, lam, dist, rvecs, tau = cv.calibrateCamera(opoints, ipoints, imgs[0].shape[::-1], None, None)

        omega = cv.Rodrigues(rvecs[0])
        tau = tau[0]
        IO.save_calibration_results([lam, dist, omega, rvecs, tau])

        return
    
    def get_camera_pose_plot(self):
        """Get camera pose visualization"""
        if not self.calibration_results:
            return None
        
        return Render.create_camera_pose_plot(
            self.calibration_results['rvecs'],
            self.calibration_results['tvecs']
        )
    
    def get_sample_images_with_axes(self, num_samples=8):
        """Get sample images with coordinate axes drawn"""
        if not self.calibration_results or not self.processed_images:
            return [None] * num_samples
        
        images = []
        for i in range(min(num_samples, len(self.processed_images))):
            img = Img.load_image(self.processed_images[i])
            
            
            img_with_axes = Render.draw_coordinate_axes(
                img,
                self.calibration_results['rvecs'][i],
                self.calibration_results['tvecs'][i],
                self.calibration_results['mtx'],
                self.calibration_results['dist']
            )

            images.append(Img.bgr_to_rgb(img_with_axes))
        
        # Pad with None if needed
        while len(images) < num_samples:
            images.append(None)
        
        return images
    
    def get_undistortion_preview(self):
        """Get undistortion before/after comparison"""
        if not self.calibration_results or not self.processed_images:
            return None, None
        
        img = Img.load_image(self.processed_images[0])
        undistorted = Cam.undistort_image(
            img, 
            self.calibration_results['mtx'], 
            self.calibration_results['dist']
        )
        
        return Img.bgr_to_rgb(img), Img.bgr_to_rgb(undistorted)

In [22]:
# Create workflow instance
workflow = CalibrationWorkflow()

# Gradio interface
with gr.Blocks(title="Camera Calibration") as demo:
    gr.Markdown("#Camera Calibration Interface")
    gr.Markdown("Upload chessboard images and run calibration with visualization")
    
    with gr.Row():
        files = gr.File(file_count="multiple", file_types=[".jpeg"])
        process_btn = gr.Button("Process Images")
    
    process_status = gr.Textbox(label="Processing Status")
    
    with gr.Tabs():
        with gr.Tab("Camera Poses"):
            pose_btn = gr.Button("Show Camera Poses")
            pose_plot = gr.Plot()
        
        with gr.Tab("Sample Images"):
            sample_btn = gr.Button("Show Images with Axes")
            with gr.Row():
                img1 = gr.Image(label="Sample 1")
                img2 = gr.Image(label="Sample 2")
                img3 = gr.Image(label="Sample 3")
                img4 = gr.Image(label="Sample 4")
                img5 = gr.Image(label="Sample 5")
        
        with gr.Tab("Undistortion"):
            undist_btn = gr.Button("Show Undistortion Preview")
            with gr.Row():
                original = gr.Image(label="Original")
                undistorted = gr.Image(label="Undistorted")
    
    # Event handlers
    process_btn.click(
        workflow.process_images, 
        inputs=[files], 
        outputs=[process_status]
    )
    
    pose_btn.click(
        workflow.get_camera_pose_plot, 
        outputs=[pose_plot]
    )
    
    sample_btn.click(
        workflow.get_sample_images_with_axes, 
        outputs=[img1, img2, img3, img4, img5]
    )
    
    undist_btn.click(
        workflow.get_undistortion_preview, 
        outputs=[original, undistorted]
    )

# Launch interface
app = demo.launch(share=True, debug=False)

* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://9f12484940e0afbfca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


./content/images/image_18.jpeg
./content/images/image_19.jpeg


Traceback (most recent call last):
  File "/home/addk3/anaconda3/envs/vision/lib/python3.13/site-packages/gradio/queueing.py", line 667, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "/home/addk3/anaconda3/envs/vision/lib/python3.13/site-packages/gradio/route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "/home/addk3/anaconda3/envs/vision/lib/python3.13/site-packages/gradio/blocks.py", line 2274, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "/home/addk3/anaconda3/envs/vision/lib/python3.13/site-packages/gradio/blocks.py", line 1781, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^